In [731]:
import matplotlib.pyplot as plt
%matplotlib inline

In [732]:
import torch

In [733]:
words = open('../names.txt', 'r').read().splitlines()

In [734]:
chars = sorted(list(set(''.join(['.'] + words))))

stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}

In [735]:
xs,  ys = [], []

for word in words:
  chs = ['.'] * 2 + list(word) + ['.'] * 2
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append([ix1, ix2])
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

number of examples:  520358


In [736]:
import torch.nn.functional as F

In [737]:
g = torch.Generator().manual_seed(2147483647)

In [738]:
W = torch.randn((27*2, 27), generator=g, requires_grad=True)

In [739]:
from math import floor
gradient_descent_count = 50
learning_rate = 50
for k in range(gradient_descent_count):

  # forward pass
  xenc= F.one_hot(xs, num_classes=27).float()
  xenc_flat = xenc.flatten(1)
  logits = xenc_flat @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(floor(num / 2)), ys].log().mean() + 0.01**(W**2).mean()

  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()

  # update step
  W.data += -learning_rate * W.grad
print(loss.item())

2.13441801071167


In [750]:
for i in range(5):

  out = []
  ix1 = 0
  ix2 = 0
  while True:
    # unsure on this
    xenc = F.one_hot(torch.tensor([ix1, ix2]), num_classes=27).float()
    xenc_flat = xenc.flatten(0)
    logits = xenc_flat @ W # predict log-counts
    # time to soft-max
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(0, keepdims=True) # probabilities for next character
    
    ix1 = ix2
    ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix2])
    if ix2 == 0:
      break
  print(''.join(out))

maelyyliibrjazia.
piontt.
ch.
duj.
ryebovilahedricosebybrewlen.


# Evaluation of trigram neural network

loss: 2.13

Improvement compared to bigram model: 2.45-2.13 = 0.32

compared to trigram counts: 2.13-1.94 = 0.19

The neural network trigram model does a little bit worse than the trigram model based on counts, but it is still better than the bigram model.